# Self Consistency Prompting

One of the more advanced techniques in prompt engineering is self-consistency, introduced by `Wang et al. (2022)`. 

This method seeks to improve upon the traditional greedy decoding typically used in chain-of-thought (CoT) prompting. 

The core concept involves sampling multiple diverse reasoning paths through few-shot CoT and leveraging these variations to determine the most consistent answer. The technique  enhances the effectiveness of CoT prompting, particularly for tasks requiring arithmetic and commonsense reasoning.

## References:
* [Wang et al. (2022)](https://arxiv.org/abs/2203.11171)

## Running this code on MyBind.org

Note: remember that you will need to **adjust CONFIG** with **proper URL and API_KEY**!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/GenILab-FAU/prompt-eng/HEAD?urlpath=%2Fdoc%2Ftree%2Fprompt-eng%2Fself_consistency.ipynb)



In [10]:
############################################################
## SELF-CONSISTENCY PROMPTING: PROJECT OVERVIEW
############################################################

import os
import csv
import random
from datetime import datetime
from _pipeline import create_payload, model_req

# 1) Define a base prompt for multiple solutions
BASE_PROMPT = """
You are an AI that generates a structured "Project Overview" section in Markdown.
The project details are:
- Title: "Automated Text Summarizer"
- Goal: "Reduce the time to create concise summaries by 50%"
- Problem Statement: "Manual summarization is slow and prone to errors"
- Key Objectives:
  1) Implement an advanced summarization algorithm
  2) Integrate user feedback loops
- Scope: "Primarily text documents up to 10 pages"

Please create a clear Markdown overview covering Title, Goal, Problem Statement, Key Objectives, and Scope.
"""

# 2) Generate multiple solutions
def generate_solutions(prompt, model_name="qwen2", num_solutions=3):
    """
    Creates multiple solutions by calling the model multiple times
    with slight variations (e.g., random temperature).
    """
    solutions = []
    for i in range(num_solutions):
        # Optionally vary temperature slightly
        temp = round(random.uniform(0.6, 0.8), 2)
        payload = create_payload(
            target="open-webui",
            model=model_name,
            prompt=prompt,
            temperature=temp,
            num_ctx=150,
            num_predict=200
        )
        time_taken, response = model_req(payload=payload)
        solutions.append({
            "attempt": i + 1,
            "temperature": temp,
            "time_taken": time_taken,
            "response": response
        })
    return solutions

# 3) Self-consistency check: re-prompt the model to choose the best solution
def pick_best_solution(solutions, model_name="qwen2"):
    """
    Sends all solutions back to the model, asking it to pick the best one.
    """
    # Build a comparison prompt
    comparison_prompt = "Below are several candidate outputs for the Project Overview:\n\n"
    for s in solutions:
        comparison_prompt += f"---\nSolution {s['attempt']} (temp={s['temperature']}):\n{s['response']}\n\n"
    comparison_prompt += """
Please analyze each solution for:
1) Clarity and completeness
2) Accuracy regarding the given project details
3) Proper Markdown formatting

Choose the best solution and explain why briefly, then provide the chosen solution in full.
"""

    payload_compare = create_payload(
        target="open-webui",
        model=model_name,
        prompt=comparison_prompt,
        temperature=0.7,
        num_ctx=250,
        num_predict=300
    )
    time_compare, response_compare = model_req(payload=payload_compare)
    return time_compare, response_compare

# 4) Generate multiple solutions
solutions = generate_solutions(BASE_PROMPT, model_name="qwen2", num_solutions=3)

# 5) Ask the model to pick the best solution
time_best, best_solution_response = pick_best_solution(solutions, model_name="qwen2")

# 6) Print out the final recommended solution
print("===== SELF-CONSISTENCY BEST SOLUTION =====")
print(best_solution_response)
if time_best:
    print(f"\nTime taken for best-solution analysis: {time_best}s")

# 7) (Optional) Log solutions and best pick
os.makedirs("data", exist_ok=True)

# Log each solution
with open("data/self_consistency_solutions.csv", "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # Optionally write a header row if needed:
    # writer.writerow(["timestamp", "attempt", "temperature", "time_taken", "response"])
    for s in solutions:
        writer.writerow([
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            s["attempt"],
            s["temperature"],
            s["time_taken"],
            s["response"].replace("\n", "\\n")
        ])

# Log the best solution
with open("data/self_consistency_best.csv", "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "best_solution",
        time_best,
        best_solution_response.replace("\n", "\\n")
    ])

# 8) Print each solution for clarity
for s in solutions:
    print(f"\nSolution {s['attempt']} (temp={s['temperature']}):")
    print(s['response'])


Payload: {'model': 'qwen2', 'messages': [{'role': 'user', 'content': '\nYou are an AI that generates a structured "Project Overview" section in Markdown.\nThe project details are:\n- Title: "Automated Text Summarizer"\n- Goal: "Reduce the time to create concise summaries by 50%"\n- Problem Statement: "Manual summarization is slow and prone to errors"\n- Key Objectives:\n  1) Implement an advanced summarization algorithm\n  2) Integrate user feedback loops\n- Scope: "Primarily text documents up to 10 pages"\n\nPlease create a clear Markdown overview covering Title, Goal, Problem Statement, Key Objectives, and Scope.\n'}]}
Payload: {'model': 'qwen2', 'messages': [{'role': 'user', 'content': '\nYou are an AI that generates a structured "Project Overview" section in Markdown.\nThe project details are:\n- Title: "Automated Text Summarizer"\n- Goal: "Reduce the time to create concise summaries by 50%"\n- Problem Statement: "Manual summarization is slow and prone to errors"\n- Key Objectives: